# Prepare data

First we will create a dictionary of Semcor words, and look at them and their frequencies.


Next, we want to create a dataset of a subsample of semcor. We want to remove the most common and least common words

We also want to limit to 50 examples of each sense of a word.

So, we begin iterating through a randomly shuffled semcor. For each word, we throw it out if it is too frequent or too infreqient. Then, we look at the senses.



At the end, we store a list of all of the words we've collected. For each item in the dictionary, we should know:
- the number of tokens
- the wordnet senses
- a list of the semcor sentence indices of the tokens of each word. 


In [1]:
import sys
sys.path.append("../lib/")

from nltk.corpus import semcor
from nltk.tree import Tree
import itertools
import random
import pandas as pd
import torch
from bert import *
import csv
from nltk.corpus.reader.wordnet import Lemma
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

import inflect
import os
from scipy.stats import spearmanr, pearsonr
import re


bert = BERTBase()


INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/9m/vzvx58rs51v_x5nm620fz4xr0000gn/T/tmpafbdoudi
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 3

In [2]:
p = inflect.engine()
p.plural("mouse")

'mice'

In [103]:
"""
load semcor stats
"""

# uncomment for whole dataset
# sents = semcor.sents()
# tagged_sents = semcor.tagged_sents( tag = ' sem ' )
# words = semcor.words()


##########
# DEBUG ONLY
############

tagged_sents = semcor.tagged_sents( tag = ' sem ' )[:20]
sents = semcor.sents()[:20]
words = semcor.words()[:1000]





In [104]:
# def get_senses_in_tagged_sentence(tagged_sentence):
#     res = []
#     for chunk in tagged_sentence:
#         chunk_string = ' '.join(chunk.leaves())

#         """
#         if we find a wordnet sense (function words dont)
#         then scoop it up

#         """            
#         if isinstance(chunk.label() , Lemma):
#             res.append(chunk.label())
#     # if we get to the end of the loop. we didn't find the word we were looking for
#     return res


In [105]:
# print(sents[1])
# get_senses_in_tagged_sentence(tagged_sents[1])

In [106]:
lemmatizer = WordNetLemmatizer()


In [109]:
lemmatizer.lemmatize("impressed")

'impressed'

In [107]:
"""
create a dictionary of tokens in semcor with counts
"""

"""
TODO is this a problem? that we have things like `group.n.01.group' which have wildly different tokens.
"""
from collections import Counter

semcor_lexicon = Counter()

for word in words:
    word = word.lower()
    lemma = lemmatizer.lemmatize(word)
    semcor_lexicon[lemma] +=1

# for sent in tagged_sents:
#     sent_lemmas = get_senses_in_tagged_sentence(sent)
#     for lemma in sent_lemmas:
#         semcor_lexicon[lemma] += 1
        
        
semcor_lexicon

Counter({'the': 82,
         'fulton': 14,
         'county': 8,
         'grand': 4,
         'jury': 19,
         'said': 14,
         'friday': 2,
         'an': 4,
         'investigation': 1,
         'of': 36,
         'atlanta': 7,
         "'s": 12,
         'recent': 1,
         'primary': 3,
         'election': 6,
         'produced': 1,
         '``': 24,
         'no': 1,
         'evidence': 1,
         "''": 23,
         'that': 16,
         'any': 1,
         'irregularity': 2,
         'took': 2,
         'place': 3,
         '.': 39,
         'further': 1,
         'in': 18,
         'term': 3,
         'end': 2,
         'presentment': 1,
         'city': 9,
         'executive': 1,
         'committee': 2,
         ',': 34,
         'which': 7,
         'had': 2,
         'over-all': 1,
         'charge': 2,
         'deserves': 1,
         'praise': 1,
         'and': 27,
         'thanks': 1,
         'for': 10,
         'manner': 2,
         'wa': 6,
         'co

In [5]:
"""
little script on semcor_lexicon to see where we should cut off most and least frequent
"""

print(semcor_lexicon.most_common(200))
n = 30000
print(semcor_lexicon.most_common()[:-n-1:-1])

# we want to keep words with a count < 600

# and with a count greater than > 10 (which is knocking off the l)

[('the', 49074), (',', 40614), ('.', 34716), ('of', 24954), ('and', 19960), ('to', 18089), ('a', 16399), ('in', 15012), ('that', 7541), ('is', 7228), ('he', 6812), ('was', 6773), ('for', 6574), ('it', 6375), ('``', 6169), ("''", 6097), ('with', 5145), ('as', 4953), ('on', 4818), ('his', 4604), ('be', 4552), ("'s", 4037), ('at', 3765), ('by', 3688), ('this', 3639), ('i', 3589), ('had', 3534), ('not', 3247), ('are', 3171), ('but', 3076), ('from', 3032), ('-', 2967), ('or', 2917), ('have', 2811), ('they', 2759), ('an', 2648), ('you', 2629), ('which', 2495), ('one', 2359), ('were', 2319), ('all', 2121), ('would', 2052), ('there', 2019), (';', 1997), ('we', 1972), ('she', 1968), ('their', 1926), ('her', 1909), ('has', 1857), ('him', 1801), ('?', 1715), ('been', 1714), ('when', 1629), ('will', 1611), ('more', 1592), ('who', 1580), ('if', 1545), (')', 1535), ('(', 1524), ('no', 1520), ('out', 1488), ('said', 1412), ("n't", 1407), ('so', 1384), ('can', 1379), ('what', 1377), ('do', 1358), ('up

In [6]:
"""
get basic semcor stats
"""
print("number of sentences:")
print(len(sents))
print("number of tokens:")
print(len(words))
print("number of types:")
print(len(semcor_lexicon))

number of sentences:
37176
number of tokens:
820411
number of types:
38642


# Step 2: Create Token Index


In [85]:
"""
Next step is to create an index of all of the tokens of a single lemma. 
So, we build a data structure with all of the word forms found in semcor. With each word form,
we store a list of all of the sentences containing it.
"""
word_index = {}
sense_index = {}

semcor_indices = list(range(0,len(sents)))
#print(semcor_indices)
random.shuffle(semcor_indices)
#print(semcor_indices)


# go through the dataset sentence by sentence
for random_index in semcor_indices:
    sent = tagged_sents[random_index]
    #sentence_id = int(sent.num)
    sentence_id = random_index
    
    # go through each sentence word by word
    for word in sent:
        senses = get_senses_in_tagged_sentence(tagged_sents[1])
        for sense in senses:
            # if this is our first time seeing this word, add it to the index and put the sentence id in the entry
            if sense not in word_index:
                word_index[sense] = {sentence_id}
            # otherwise, add the sentence id to the entry for the word
            else:
                word_index[sense].add(sentence_id)
        
#     # we need to make sure we are collecting only those tokens which have semcor senses, or we make note of which ones do
    
#         # if this is our first time seeing this word, add it to the index and put the sentence id in the entry
#         if word not in word_index:
#             word = word.lower()
#             word_index[word] = {sentence_id}
#         # otherwise, add the sentence id to the entry for the word
#         else:
#             word_index[word].add(sentence_id)
            

In [86]:
"""
let's take a look at it
"""
print(list(word_index.items())[:20])

[(Lemma('jury.n.01.jury'), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}), (Lemma('far.r.02.far'), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}), (Lemma('state.v.01.say'), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}), (Lemma('term.n.02.term'), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}), (Lemma('end.n.02.end'), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}), (Lemma('presentment.n.01.presentment'), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}), (Lemma('group.n.01.group'), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}), (Lemma('own.v.01.have'), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}), (Lemma('overall.s.02.overall'), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}), (Lemma('mission.n.03.charge'), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1

In [116]:
render1 = wn.lemma('render.v.07.return')
# render2 = wn.lemma('return.v.01.return')

# """
# importnt point about nltk wordnet lemmas. their representation is confusing so be careful. i think equals or differentequals are implmementd in
# unsuspected ways, because you get issues where they dont act like their display name
# """

# dixt = {str(render1): "foo"}
# dixt[str(render2)] = "bar"
# print(dixt)


# dixt = {str(render1): "foo"}
# dixt[str(render2)] = "bar"
# print(dixt)

re.findall(r"\.(.*?)\.", 'render.v.07.return')


['v']

In [10]:
"""
Now that we have our word index, we want to construct the evaluation dataset

for each word in the index, we want

We iterate through the words in the dictionary.
if that word has a frequency of  > 600 or < 7, we move on.
If it has the right frequency, we pull the indices of the sentences containing that word
we shuffle these indices and access in random order. 
We go through the shuffled indices,
    and we check if we have collected < 50 of this sense.
    if not, we collect this token for the evaluation dataset

collection means:
    we construct a row of data like
        word sense
        word form
        token
        
        
at the end we save the data in a csv file called 'semcor_wu_palmer_eval_data.csv'
"""

def get_sense_in_tagged_sentence(word, tagged_sentence):
    for chunk in tagged_sentence:

        chunk_string = ' '.join(chunk.leaves())

        """
        if we find the word we're looking for in this chunk,
        and that chunk has a wordnet sense (function words dont)
        then scoop it up

        """            
        if chunk_string.lower() == word:
            #print("found %s" % word)
            #print(chunk.label())

            #wn_lemma = cunk.label()
            if isinstance(chunk.label() , Lemma):
                return chunk.label()
    # if we get to the end of the loop. we didn't find the word we were looking for
    return None


def collect_tokens(word, indices, sents, tagged_sents):
    """
    takes a word and a list of indices
    returns tuples containing 
        word
        sentence_string
        sense
    """
    sense_count = Counter()

    print("collecting tokens for ", word)
    print("indices:   ", indices)
    tokens = []
    
    # indices is a list of all of the sentence ids containing this word
    indices = list(indices)
    # visit these sentences in random order
    random.shuffle(indices)
    for index in indices:
        
        #print(word)
        #print(index)
        tagged_sentence = tagged_sents[index]
        #print(tagged_sentence)
        sentence = sents[index]
        #print(sentence)

        sentence = ' '.join(sentence)
        sense = get_sense_in_tagged_sentence(word, tagged_sentence)
        #print(sense)
        
        if sense is None:
            # we have a word in the sentence matching this word form but it doesnt constitute a chunk on its own;
            # part of some MWE.
            continue
        
        #print(sense.name())
        
        sense = str(sense)
        if sense_count[sense] <= 25:
            tokens.append((word, sentence, sense))
            sense_count[sense] += 1
    
    print(sense_count.items())
    return tokens, sense_count


def collect_tokens_for_all_words_to_file(path, sense_path, word_index, semcor_lexicon, sents, tagged_sents):
    with open(path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        with open(sense_path, 'w', newline='') as sensefile:
            sensewriter = csv.writer(sensefile)
        
            for word, indices in word_index.items():
                #print(word)
                #print(indices)

                frequency = semcor_lexicon[word]
                if frequency < 600 and frequency > 7:
                    rows, sense_count = collect_tokens(word, indices, sents, tagged_sents)

                    writer.writerows(rows)                    
                    sensewriter.writerow(sense_count.items())
                


collecting tokens for  performances
indices:    {25991, 31625, 15755, 26635, 31631, 1042, 2458, 1056, 32674, 301, 1722, 26301, 1726, 25663, 11198, 26559, 22092, 26317, 26335, 26976, 26344, 26473, 26352, 22385, 37116}
dict_items([("Lemma('performance.n.02.performance')", 3), ("Lemma('performance.n.03.performance')", 1), ("Lemma('performance.n.01.performance')", 3)])
collecting tokens for  assured
indices:    {23296, 36224, 16898, 910, 20370, 31763, 31764, 2199, 26009, 22042, 24091, 25759, 2594, 30758, 15148, 14263, 9144, 31320, 25051, 25447, 26473, 7403, 26352, 18547, 20085, 11127}
dict_items([("Lemma('assure.v.03.assure')", 5), ("Lemma('guarantee.v.02.assure')", 5), ("Lemma('assure.v.02.assure')", 5), ("Lemma('reassure.v.01.assure')", 1), ("Lemma('assured.s.01.assured')", 2), ("Lemma('promise.v.01.assure')", 1), ("Lemma('assured.s.02.assured')", 1), ("Lemma('see.v.10.assure')", 1)])
collecting tokens for  informal
indices:    {22598, 11432, 22729, 25418, 14059, 13356, 13357, 27852, 263

dict_items([("Lemma('allege.v.01.say')", 15), ("Lemma('state.v.01.say')", 26), ("Lemma('suppose.v.01.say')", 3)])
collecting tokens for  harsh
indices:    {20234, 12267, 12908, 27088, 18642, 36051, 2551, 28411}
dict_items([("Lemma('harsh.s.02.harsh')", 2), ("Lemma('harsh.s.01.harsh')", 2)])
collecting tokens for  word
indices:    {10633, 23851, 26448, 17847, 28380}
dict_items([("Lemma('word.n.01.word')", 2)])
collecting tokens for  showed
indices:    {4097, 3586, 3588, 4103, 17928, 4108, 4111, 4117, 4118, 3100, 3102, 1055, 3114, 3115, 4152, 26170, 4155, 24125, 3137, 8259, 14407, 584, 10315, 32845, 4182, 35928, 3164, 34911, 4199, 27249, 3713, 651, 19089, 27288, 17562, 5283, 4261, 14503, 37031, 26791, 4268, 37037, 9903, 19635, 10932, 6846, 25280, 31457, 6888, 21231, 31472, 37107, 33526, 28407, 13579, 18715, 36644, 19253, 6966, 4921, 17723, 27975, 18760, 13131, 10573, 2388, 18786, 28003, 26477, 21871, 5493, 375, 887, 30590, 2947, 36239, 2971, 3996, 12699, 35746, 12196, 15786, 7595, 4021, 

dict_items([("Lemma('largely.r.01.largely')", 23)])
collecting tokens for  mobile
indices:    {23693, 13351, 28458, 28459, 28463, 28470, 13367, 28473, 22076, 28477, 28478, 28480, 34762, 28509, 31596, 28525, 34671, 26742, 32380}
dict_items([("Lemma('mobile.s.01.mobile')", 2)])
collecting tokens for  packed
indices:    {1698, 33378, 3077, 21358, 9326, 34127, 27921, 30899, 30419, 4120, 22076, 1534, 17535}
dict_items([("Lemma('pack.v.01.pack')", 6), ("Lemma('pack.v.03.pack')", 2), ("Lemma('pack_on.v.01.pack_on')", 1), ("Lemma('jammed.s.01.packed')", 1), ("Lemma('pack.v.02.pack')", 1)])
collecting tokens for  moment
indices:    {25603, 25604, 35847, 10764, 18449, 35347, 36888, 6170, 21531, 35870, 6177, 24098, 1058, 31267, 5672, 1576, 23096, 33850, 3131, 22076, 8253, 33359, 33363, 2132, 2645, 36952, 26713, 2652, 33373, 34404, 25700, 26215, 17000, 7273, 2664, 2160, 35444, 18037, 17013, 33912, 30330, 10368, 1155, 20100, 34438, 13447, 18569, 11916, 5773, 31886, 8847, 25747, 29335, 17048, 5271, 

dict_items([("Lemma('corner.n.02.corner')", 9), ("Lemma('corner.n.05.corner')", 2), ("Lemma('corner.n.01.corner')", 10), ("Lemma('corner.n.03.corner')", 8), ("Lemma('corner.n.04.corner')", 6), ("Lemma('recess.n.02.corner')", 2), ("Lemma('corner.n.07.corner')", 1), ("Lemma('corner_post.n.01.corner_post')", 1), ("Lemma('cut_corners.v.01.cut_corners')", 1)])
collecting tokens for  day
indices:    {36961, 35939, 27461, 17926, 24777, 23536, 14615}
dict_items([("Lemma('day.n.03.day')", 1), ("Lemma('day.n.01.day')", 1)])
collecting tokens for  gabriel
indices:    {34339}
dict_items([])
collecting tokens for  horn
indices:    {26618, 18244}
dict_items([])
collecting tokens for  devil
indices:    {1273, 36236, 28287}
dict_items([("Lemma('satan.n.01.Devil')", 1)])
collecting tokens for  random
indices:    {15905, 4456, 22095, 16181, 5530, 4476}
dict_items([("Lemma('random.a.01.random')", 1), ("Lemma('random-access_memory.n.01.random-access_memory')", 1)])
collecting tokens for  thrown
indices:  

dict_items([("Lemma('history.n.01.history')", 8), ("Lemma('history.n.04.history')", 4), ("Lemma('history.n.02.history')", 3), ("Lemma('history.n.05.history')", 1), ("Lemma('history.n.03.history')", 2)])
collecting tokens for  usefulness
indices:    {2304, 31234, 16134, 16135, 16137, 24172, 12236, 22774}
dict_items([("Lemma('utility.n.02.usefulness')", 5)])
collecting tokens for  nationalism
indices:    {2296, 31235, 31236, 16420, 31224, 4632, 27753, 31245, 16430, 27731, 2294, 31222, 27736, 31227}
dict_items([("Lemma('nationalism.n.02.nationalism')", 2), ("Lemma('patriotism.n.01.nationalism')", 3)])
collecting tokens for  territorial
indices:    {16161, 27015, 31719, 31180, 31182, 31224}
dict_items([("Lemma('territorial.a.01.territorial')", 1)])
collecting tokens for  rapidly
indices:    {31234, 37125, 17545, 25737, 18833, 2837, 13334, 15515, 1533, 18467, 18855, 5544, 3500, 31025, 16434, 3634, 13108, 13365, 15158, 27954, 32313, 22714, 4793, 16316, 5568, 23621, 2631, 20553, 8907, 14164, 

...

dict_items([("Lemma('mentally.r.01.mentally')", 5)])
collecting tokens for  alveolar
indices:    {3808, 3817, 3850, 3793, 3410, 3794, 3770, 3803, 3807}
dict_items([("Lemma('alveolar.a.01.alveolar')", 8)])
collecting tokens for  straw
indices:    {10497, 706, 6500, 8103, 9512, 13575, 9325, 9326, 26704, 9361, 18480, 9494, 1657, 29982}
dict_items([("Lemma('straw.n.01.straw')", 8), ("Lemma('straw.s.01.straw')", 1), ("Lemma('chaff.n.01.straw')", 1)])
collecting tokens for  suitcase
indices:    {34144, 10497, 34136, 16937, 34154, 34155, 34158, 34127, 17199, 17138, 17139, 34132, 9944, 15741}
dict_items([("Lemma('bag.n.06.suitcase')", 7)])
collecting tokens for  catcher
indices:    {19842, 19811, 19816, 19833, 19818, 19793, 22995, 19832, 185, 186, 19804, 19806}
dict_items([("Lemma('catcher.n.01.catcher')", 11)])
collecting tokens for  egg
indices:    {3616, 9337, 3618, 3655, 3624, 1706, 30413, 9394, 29236, 3641}
dict_items([("Lemma('egg.n.01.egg')", 6)])
collecting tokens for  disturbed
indice

dict_items([("Lemma('point_of_view.n.01.viewpoint')", 10)])
collecting tokens for  bull's-eye
indices:    {4387, 777, 4394, 4460, 780, 4430, 4401, 4435, 4437}
dict_items([("Lemma('bull's_eye.n.01.bull's_eye')", 6), ("Lemma('bell_ringer.n.03.bull's_eye')", 1), ("Lemma('bull's_eye.n.02.bull's_eye')", 2)])
collecting tokens for  eagerly
indices:    {17764, 30821, 12015, 27025, 35027, 15827, 4762}
dict_items([("Lemma('eagerly.r.01.eagerly')", 4)])
collecting tokens for  skiff
indices:    {34434, 34436, 8648, 34447, 34421, 34422, 34426, 34428, 8573, 8639}
dict_items([("Lemma('skiff.n.01.skiff')", 3)])
collecting tokens for  purchasing
indices:    {11777, 20315, 23555, 32453, 6, 2737, 15230, 32305, 2772, 24054, 24057, 2779, 2750, 31775}
dict_items([("Lemma('buy.v.01.purchase')", 7)])
collecting tokens for  1945
indices:    {5120, 12709, 12933, 20585, 25418, 21421, 25263, 26191, 31636, 3999}
dict_items([])
collecting tokens for  prosperity
indices:    {24033, 28098, 12917, 25176, 27769}
dict_

dict_items([("Lemma('support.n.08.supporting')", 1), ("Lemma('back.v.01.support')", 1), ("Lemma('hold.v.10.support')", 2), ("Lemma('support.v.02.support')", 1), ("Lemma('support.v.01.support')", 3), ("Lemma('encouraging.s.02.supporting')", 2)])
collecting tokens for  gradual
indices:    {13216, 31808, 31850, 23476, 4022, 30807}
dict_items([("Lemma('gradual.a.01.gradual')", 2)])
collecting tokens for  cereal
indices:    {30448, 30450, 30451, 30452, 30454, 30455, 30488, 37081, 30493}
dict_items([])
collecting tokens for  trigger
indices:    {18822, 31304, 18408, 29068, 6925, 29070, 18609, 35573, 5910, 31256, 29113}
dict_items([("Lemma('trip.v.04.trigger')", 1), ("Lemma('gun_trigger.n.01.trigger')", 4)])
collecting tokens for  satire
indices:    {10858, 10859, 1006, 10832, 14482, 27027, 11187, 10805}
dict_items([("Lemma('sarcasm.n.01.satire')", 7)])
collecting tokens for  await
indices:    {5472, 23589, 2570, 2559, 7695, 3759, 4991}
dict_items([("Lemma('expect.v.03.await')", 7)])
collecti

dict_items([("Lemma('pseudo.s.01.pseudo')", 7)])
collecting tokens for  harvest
indices:    {13973}
dict_items([])
collecting tokens for  promoted
indices:    {10114, 1191, 15216, 11892, 15221, 10167, 33049, 28091, 31614}
dict_items([("Lemma('promote.v.01.promote')", 3), ("Lemma('promote.v.02.promote')", 5), ("Lemma('advertise.v.02.promote')", 1)])
collecting tokens for  ditch
indices:    {18945, 33826, 5059, 33806, 5522, 5558, 33848, 33823}
dict_items([("Lemma('ditch.n.01.ditch')", 2)])
collecting tokens for  bow
indices:    {30890, 21252, 9430}
dict_items([("Lemma('bow.n.01.bow')", 1), ("Lemma('submit.v.06.bow')", 1)])
collecting tokens for  null
indices:    {4289, 4290, 32099, 4292, 14822, 4328, 4298, 4366, 4371, 4314}
dict_items([("Lemma('null.s.01.null')", 1)])
collecting tokens for  hemphill
indices:    {20669}
dict_items([])
collecting tokens for  nilpotent
indices:    {4352, 4355, 4357, 4336, 4339, 4340, 4341, 4342, 4348, 4349, 4351}
dict_items([("Lemma('nilpotent.a.01.nilpoten

dict_items([("Lemma('kernel.n.03.essence')", 2)])
collecting tokens for  coins
indices:    {18949, 31945, 17581, 21616, 21618, 30709, 31898, 14719}
dict_items([("Lemma('coin.n.01.coin')", 3)])
collecting tokens for  pursuit
indices:    {5220, 36010, 22699, 32076, 29964, 25198, 36911, 21330, 31989, 13695}
dict_items([("Lemma('pursuit.n.02.pursuit')", 1), ("Lemma('pursuit.n.01.pursuit')", 1)])
collecting tokens for  gene
indices:    {12641}
dict_items([])


In [ ]:

# DEBUG TEST
#collect_tokens('heart', word_index['heart'], sents, tagged_sents)

collect_tokens_for_all_words_to_file('../data/semcor_wu_palmer_eval_data.csv', '../data/semcor_sense_counts.csv', word_index, semcor_lexicon, sents, tagged_sents)

In [11]:
print(len(sents))
print(len(tagged_sents))
print(len(word_index))
print(len(semcor_lexicon))

37176
37176
38642
38642


In [12]:
"""
models and save paths

"""
models = [
    'trained_models/model.plsr.buchanan.allbuthomoyms.5k.300components.500max_iters',
    'trained_models/model.plsr.buchanan.allbuthomoyms.1k.300components.500max_iters',
    #'trained_models/model.plsr.buchanan.allbuthomoyms.glove.300components.300max_iters',
    'trained_models/model.ffnn.buchanan.allbuthomoyms.5k.50epochs.0.5dropout.lr1e-4.hsize300',
    'trained_models/model.ffnn.buchanan.allbuthomoyms.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    #'trained_models/model.ffnn.buchanan.allbuthomoyms.glove.50epochs.0.5dropout.lr1e-4.hsize300',
    'trained_models/model.modabs.buchanan.allbuthomoyms.5k',
    'trained_models/model.modabs.buchanan.allbuthomoyms.1k',
    #'trained_models/model.modabs.buchanan.allbuthomoyms.glove'
]

In [17]:
"""
Now, we have our dataset that we want to analyze. We just need to do:

for each model we want to evaluate, run the following script:

open the file of data

read it in as a dataframe

for each of the unique words in that dataset

    we calculate pairwise distances between each otoken and every otehr token
    and construct a similarities dataset. 
    
    then we run correlations for that word???
    and store into a file
"""

def lemma_from_string(lemma_string):
    # grabs everything in between (' ') in a string
    # (needed to update from r"'(.*?)'" to deal with cases with quotes in word like o'clock)
    string = re.findall(r"\('(.*?)'\)", lemma_string)[0]
    #print(string)
    lemma = wn.lemma(string)
    return lemma

def make_predictions(df, model, bert):
    predictions = []
    for index, row in df.iterrows():

        #print(row.word_form)
        #print(row.context)

        predicted_vector = model.predict_in_context(row.word_form, row.context, bert)

        predictions.append(predicted_vector)
    return predictions

def get_pairwise_wu_palmer_data(model, df, bert, outfile):
    unique_words = df.word_form.unique()
    
    #run_stats = [0] * len(unique_words)
    run_stats = []

    for i in range(0, len(unique_words)):
        if i % 500 == 0:
            print("processed %s words" % i)
        
        # a dataframe containing all the tokens of this word
        word = unique_words[i]
        word_data = df[df.word_form == word].copy()

        n_senses = len(word_data['wn_lemma'].unique())

        predictions = make_predictions(word_data, model, bert)
        
        word_data['prediction'] = predictions
        
        #print(word_data)

        """

        then we calculate the pairwise distances between all of the vectors, only counting one pair one time

        """
        vals_for_this_word = []
#         cosines_for_this_word = []
#         wup_sims_for_this_word = []
#         sense_1 = []
#         sense_2 = []

        # pop the first token off the list
        num_toks = len(word_data)
        for i in range(0,num_toks):
            # compare it with each of the other tokens
            # dont have to compare to any earlier
            for j in range(i+1,num_toks):

                #print(df.iloc[i])
                #print(df.iloc[j])

                # calculate cosine similarity between the two vectors
                cos_sim = 1 - cosine(word_data.iloc[i].prediction, word_data.iloc[j].prediction)

                # and wu palmer similarity between the two wn lemmas
                lemma_1 = lemma_from_string(word_data.iloc[i].wn_lemma)
                lemma_2 = lemma_from_string(word_data.iloc[j].wn_lemma)
                synset1 = lemma_1.synset()
                synset2 = lemma_2.synset()
                wup_sim = synset1.wup_similarity(synset2)

                # if we can't compute a distance for these senses / recognize them, discard
                if type(wup_sim) == float:
                    # store this data point into a list
                    vals_for_this_word.append((word, lemma_1, lemma_2, cos_sim, wup_sim))
        
        
        token_similarities = pd.DataFrame.from_records(vals_for_this_word, columns = ["lemma", "token_sense_1", "token_sense_2", "cos_sim", "wup_sim"])
        token_similarities['n_senses'] = n_senses
        
        #print(token_similarities)
        token_similarities.to_csv(outfile, mode='a', header=False)
        
        
        # run correlation on this word
#         if sense_similarities is not None:
#             n = len(word_data)
#             n_senses = len(word_data['wn_lemma'].unique())
#             #print(word)
#             #print(n)
#             #print(sense_similarities)
#             pearson, pearson_p, spearman, spearman_p = run_correlation(sense_similarities)
#             # store the values for this word into a dataframe
#             #run_stats[i] = (word, n, pearson, pearson_p, spearman, spearman_p)
#             run_stats.append((word, n, n_senses, pearson, pearson_p, spearman, spearman_p))

    #res = pd.DataFrame.from_records(run_stats, columns = ["word", "n", "n_senses", "pearson", "pearson_p", "spearman", "spearman_p"])
    return None

    #print(all_sense_similarities)
    #return all_sense_similarities
        
        
def run_correlation(sense_similarities):
    """
    :sense_similarities: dataframe with columns
        cosine_sims 
        wup_sims
    """
    if sense_similarities is None:
        # not really sure why we're getting none values here it should be impossible
        return (float("nan"), float("nan"), float("nan"), float("nan"))
    elif len(sense_similarities['wup_sim'].unique()) == 1:
        # the correlation will be garbage with a constant y value; skip to avoid warnings
        return (float("nan"), float("nan"), float("nan"), float("nan"))

    if len(sense_similarities) > 1 :
        #print(word)

        cos_sims = sense_similarities['cos_sim']
        wup_sims = sense_similarities['wup_sim']

        pearson, pearson_p = pearsonr(cos_sims, wup_sims )
        #print('Pearsons correlation: %.3f, p-value: %s'  % (pearson, pearson_p))

        spearman, spearman_p = spearmanr(cos_sims, wup_sims )
        #print('Spearmans correlation: %.3f, p-value: %s'  % (spearman, spearman_p))

        return (pearson, pearson_p, spearman, spearman_p)
    return (float("nan"), float("nan"), float("nan"), float("nan"))

def plot_sims():
    cos_sims = sense_similarities['cos_sim']
    wup_sims = sense_similarities['wup_sim']
    plt.scatter(wup_sims, cos_sims)
    plt.title("Wordnet similarity of homonymous senses plotted against cosine similarity of predicted vectors of two tokens in semantic feature space")
    plt.xlabel("Wu and Palmer Similarity")
    plt.ylabel("Cosine Similarity")
    plt.show()
        

df = pd.read_csv('../data/semcor_wu_palmer_eval_data.csv', names = ["word_form", "context", "wn_lemma"])



for save_path in models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load("../" + save_path)
    out_path = '../results/semcor_pairwise_data_' + os.path.split(save_path)[1] + '.csv'

    # remove results file if exists
    if os.path.exists(out_path):
        os.remove(out_path)
    get_pairwise_wu_palmer_data(model, df, bert, out_path)


****************************************
*** Evaluating trained_models/model.plsr.buchanan.allbuthomoyms.5k.300components.500max_iters model ***
****************************************
processed 0 words
processed 500 words
processed 1000 words
processed 1500 words
processed 2000 words
processed 2500 words
processed 3000 words
processed 3500 words
processed 4000 words
processed 4500 words
processed 5000 words
processed 5500 words
processed 6000 words
****************************************
*** Evaluating trained_models/model.plsr.buchanan.allbuthomoyms.1k.300components.500max_iters model ***
****************************************
processed 0 words
processed 500 words
processed 1000 words
processed 1500 words
processed 2000 words
processed 2500 words
processed 3000 words
processed 3500 words
processed 4000 words
processed 4500 words
processed 5000 words
processed 5500 words
processed 6000 words
****************************************
*** Evaluating trained_models/model.ffnn.buchanan

In [31]:
brysbaert_filename = "/Users/gabriellachronis/data/Concreteness_ratings_Brysbaert_et_al_BRM.csv"
concreteness_df = pd.read_csv(brysbaert_filename, sep='\t')
concreteness_df= concreteness_df[["Word", "Conc.M"]]
concreteness_df = concreteness_df.set_index("Word")
concreteness_df.head()

,Conc.M
Word,
roadsweeper,4.85
traindriver,4.54
tush,4.45
hairdress,3.93
pharmaceutics,3.77


In [129]:
"""
Now we need to go in and add abstractness value and 
bin number of senses into polysemy band
"""



import pandas as pd

cols = ["lemma", "token_sense_1", "token_sense_2", "cos_sim", "wup_sim", "n_senses"]

for save_path in models:
    print("****************************************")
    print("*** Saturating pairwise data for model: %s ***" % save_path)
    print("****************************************")
    infile = '../results/semcor_pairwise_data_' + os.path.split(save_path)[1] + '.csv'

    outfile = '../results/saturated_semcor_pairwise_data_' + os.path.split(save_path)[1] + '.csv'

    csv_input = pd.read_csv(infile, names=cols)
    
    csv_input['wn_bin'] = pd.cut(csv_input.n_senses, 
                        bins = [0, 2.1, 4.1, 6.1, 8.1, 10.1, 20.1, 50.1, 200], labels = False)

    # add POS rows
    pos1s = []
    pos2s = []
    for index, row in csv_input.iterrows():
        pos1 = re.findall(r"\.(.*?)\.", row.token_sense_1)[0]
        pos2 = re.findall(r"\.(.*?)\.", row.token_sense_2)[0]
        pos1s.append(pos1)
        pos2s.append(pos2)
    
    csv_input['sense1_pos'] = pos1s
    csv_input['sense2_pos'] = pos2s
    
    # add concreteness
    csv_input = csv_input.join(concreteness_df, how = "left", on = "lemma")
    
    csv_input['conc_bin'] = pd.cut(csv_input['Conc.M'], 
                        bins = [0, 2.3, 4.5, 10], labels = False)
    
    # remove token sense columns
    csv_input.drop(['token_sense_1'], axis=1)
    csv_input.drop(['token_sense_2'], axis=1)
    
    #print(csv_input.where(csv_input['Conc.M'].notnull()))
    print(csv_input.head())
    #csv_input.to_csv(outfile, index=False)
    
    raise Exception("dewfieow")

****************************************
*** Saturating pairwise data for model: trained_models/model.plsr.buchanan.allbuthomoyms.5k.300components.500max_iters ***
****************************************
          lemma                          token_sense_1  \
0  performances  Lemma('performance.n.02.performance')   
1  performances  Lemma('performance.n.02.performance')   
2  performances  Lemma('performance.n.02.performance')   
3  performances  Lemma('performance.n.02.performance')   
4  performances  Lemma('performance.n.02.performance')   

                           token_sense_2   cos_sim   wup_sim  n_senses  \
0  Lemma('performance.n.03.performance')  0.364395  0.555556         3   
1  Lemma('performance.n.01.performance')  0.755421  0.444444         3   
2  Lemma('performance.n.02.performance')  0.793253  1.000000         3   
3  Lemma('performance.n.01.performance')  0.698643  0.444444         3   
4  Lemma('performance.n.02.performance')  0.749393  1.000000         3   

 

Exception: dewfieow

In [119]:
lemma_from_string(csv_input.iloc[0].token_sense_1)

Lemma('performance.n.02.performance')